# Instalando bibliotecas e requisitos necessários

Vamos instalar as bibliotecas necessárias

In [ ]:
!pip install ale-py
!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]

Com as bibliotecas instaladas, vamos agora importá-las.

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # linha para corrigir um erro que dava ao importar o torch no Jupyter Notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
import ale_py

from stable_baselines3.common.atari_wrappers import AtariWrapper
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env, make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.callbacks import CheckpointCallback

from stable_baselines3 import DQN, PPO, A2C

# Treinamento da Rede Neural

A ideia aqui é criar 20 ambientes ao mesmo tempo de modo a IA treinar simultaneamente, para acelerar o treinamento. O número de passos eu coloquei em 1,5 milhão para aumentar a perfomance. Empiricamente, vi que a partir de 700.000 passos a IA começa a ter progresso, até lá, ela estabiliza em -20 e demora a engatar.

Além disso, ao invés de mandar para a rede neural somente o frame atual, mandaremos os últimos 4 frames, usando o [VecFrameStack](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#stable_baselines3.common.vec_env.VecFrameStack), assim a IA poderá ter alguma noção da velocidade e a trajetória da bolinha

Para esse jogo em si, eu devo ter treinado algo em torno de 3 milhões de passos, usando uma GTX 1060 5GB, no total deve ter treinado o modelo por 4 horas, no acumulado.

In [ ]:
NUMERO_AMBIENTES = 20
NUMERO_FRAMES = 4
NUMERO_PASSOS = 1_000_000

gym.register_envs(ale_py) # Registramos os ambientes do Atari no Gym
def make_env(render=False): # Vamos criar uma função para criar o ambiente corretamente
        return gym.make("PongNoFrameskip-v4")
    
# Aqui criamos a vetorização do ambiente, de modo a permitir multiprocessamento
vec_env = make_atari_env(make_env, n_envs=NUMERO_AMBIENTES, seed=0)
# Usamos FrameStack para empilhar os últimos 4 frames, desse modo, a rede neural consegue ter noção de movimento e velocidade.
vec_env = VecFrameStack(vec_env, n_stack=NUMERO_FRAMES)

Criaremos o modelo.

Para esse jogo, decidi usar o algoritmo [PPO](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html).

Eu testei usar também o [DQN](https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html), só que o replaybuffer desse algoritmo tinha 15GB, o que tornou custosa sua implementação, em contraste, o arquivo para salvar o modelo do PPO tem o equivalente a 20MB. Deve-se lembrar que empiricamente o DQN tende a convergir mais rápido para esse jogo em questão do que o PPO, só que a diferença não é gigantsca e o PPO tem resultados satisfatórios, até quando comparado com outros algoritmos [como pode ser ver nesse paper](https://arxiv.org/pdf/1707.06347)
Eu otimizei os hiperparâmetros do modelo com o auxílio do ChatGPT, e cheguei ao resultado abaixo.

Vejam que no retorno aparece "cuda device", isto é porque eu já instalei o pytorch no meu ambiente conda, algo que você pode fazer [aqui](https://pytorch.org/get-started/locally/#start-locally)

In [ ]:
model = PPO(
    'CnnPolicy', 
    vec_env,
    n_steps=128,  # Número de passos por atualização
    batch_size=512,  # Tamanho do batch para atualização
    n_epochs=4,  # Número de epochs por atualização
    learning_rate=1e-4,
    clip_range=0.1,  # Menor que o padrão para mais estabilidade
    ent_coef=0.01,  # Incentiva exploração
    vf_coef=0.5,
    max_grad_norm=0.5,
    gae_lambda=0.95,
    normalize_advantage=True,
    verbose=1,
    tensorboard_log="./pong/tensorboard_ppo/"
)
model.save(path="modelo_pong_2.zip")
# model.save_replay_buffer("replay_buffer_pong.pkl")

Aqui iremos carregar o modelo criado anteriormente (ou já treinado) e iniciar (continuar) o treinamento.

Como eu já estava sempre treinando o modelo quando tinha tempo, eu executava mais essa parte do código que a anterior, porque o ambiente já foi criado na célula acima dessa e o modelo já estava salvo na minha máquina.

In [ ]:
model = PPO.load("modelo_pong.zip")
# model.load_replay_buffer("replay_buffer_pong.pkl") caso use DQN
model.set_env(vec_env)
model.learn(NUMERO_PASSOS, progress_bar=True)
model.save(path="modelo_pong.zip")
# model.save_replay_buffer("replay_buffer_pong.pkl") caso use DQN

Como você pode ver, depois milhões de passos (no acumulado de 3 sessões que fiz, devem ter dado 3 milhões de passos) o modelo já atingiu o ápice da recompensa (20), ultrapassando em muito a CPU do jogo

# Testando a Rede Neural

Iremos agora testar o modelo.

Vamos recriar o ambiente, dessa vez com modo de renderização humano, e configurando para 1 ambiente somente (não é necessário paralelismo para apenas ver o resultado)

Deve-se lembrar do seguinte: esse código só irá rodar em uma máquina local, como o Jupyter Notebook instalado pelo conda, além de ter o torch integrado com o cuda, ele não será capaz de rodar no Google Colab nem no Kaggle, para isso, eu deixei apenas o vídeo do resultado final após os entornos de 3 milhões de passos efetuados.

In [ ]:
def make_env(render=False):
        return gym.make("PongNoFrameskip-v4", render_mode="human")
env = make_atari_env(make_env, n_envs=1)  # Note n_envs=1
env = VecFrameStack(env, n_stack=4)  # Mesmo n_stack usado no treino
model = PPO.load("modelo_pong.zip")
obs = env.reset()
while True:
    action = model.predict(obs)[0]
    obs, rewards, dones, info = env.step(action)
    env.render()
    if dones:
        obs = env.reset()

Aqui uma demonstração da brutal goleada que a CPU passou pra minha IA.

In [3]:
from IPython.display import HTML
from base64 import b64encode

def play(filename):
    html = ''
    video = open(filename,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=700 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return HTML(html)

play('/kaggle/input/salomao-1/The Arcade Learning Environment 2025-01-30 19-17-48.mp4')


### Algumas observações e conclusão final

Evidentemente, eu não criei toda a rede neural do 0, não escolhi as camadas e a forma que os neurônios foram distribuídos, mas esse é apenas um passo inicial para o desenvolvimento de RL.

Além de tentar servir de inspiração pra quem tem vontade de seguir na área, também serve, para mim, como um fundamento para projetos maiores.

Pelo tempo até aqui gasto, obrigado.